# Cabecera de codigo para seleccionar el dispositivo a usar

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1" or "-1" if you want to use the cpu;
os.environ["CUDA_VISIBLE_DEVICES"]="-1";

# Importacion de Librerias

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from shutil import copy2 as copy
import multiprocessing.pool
from six.moves import range

import numpy as np

# Organizando el DataSet

In [2]:
%cd ../DataSets

if os.path.isdir("ucf101"):
    %cd ucf101
else:
    os.mkdir("ucf101")

/home/jefelitman/DataSets
/home/jefelitman/DataSets/ucf101


In [3]:
ds_directory = "/home/jefelitman/DataSets/ucf101"

In [4]:
splits_path = "/home/jefelitman/ucf101_splits"
files_path = os.listdir(splits_path)
files_path

['testlist02.txt',
 'trainlist01.txt',
 'classInd.txt',
 'trainlist03.txt',
 'testlist01.txt',
 'testlist03.txt',
 'trainlist02.txt']

In [5]:
files_path.pop(2)

'classInd.txt'

### Creacion de las carpetas de clase en los train

In [7]:
clases_file = open(os.path.join(splits_path,'classInd.txt'),"r")
clases = []
for clase in clases_file:
    clases.append(clase.split(" ")[1][:-1])
len(clases)

101

In [14]:
for split in ["split_1","split_2","split_3"]:
    split_path = os.path.join(ds_directory,split)
    for folder in os.listdir(split_path):
        for clase in clases:
            os.mkdir(os.path.join(split_path,folder,clase))

### Colocando los archivos en los splits segun corresponda

In [15]:
origen_path = "/home/jefelitman/DataSets/UCF101/frames"
for file in files_path:
    archivo = open(os.path.join(splits_path,file),"r")
    
    if file.endswith("01.txt"):
        split_path = os.path.join(ds_directory,"split_1")
    elif file.endswith("02.txt"):
        split_path = os.path.join(ds_directory,"split_2")
    elif file.endswith("03.txt"):
        split_path = os.path.join(ds_directory,"split_3")
    if file.startswith("train"):
        split_path = os.path.join(split_path,"train")
    elif file.startswith("test"):
        split_path = os.path.join(split_path,"test")
    
    for linea in archivo:
        clase = linea.split("/")[0]
        video = linea.split("/")[1][:-5]
        dir_ori_path = os.path.join(origen_path,video)
        dest = os.path.join(split_path,clase,video)
        os.mkdir(dest) #Creamos la carpeta del destino
        
        for frame in os.listdir(dir_ori_path):
            ori = os.path.join(dir_ori_path,frame)
            copy(ori,dest)

FileNotFoundError: [Errno 2] No such file or directory: '/home/jefelitman/DataSets/UCF101/frames/v_HandStandPushups_g08_c01'

In [ ]:
os.listdir("/home/jefelitman/DataSets/UCF101/frames")[:2]

In [11]:
dest

'/home/jefelitman/DataSets/ucf101/split_2/test/ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01'

In [12]:
!ls /home/jefelitman/DataSets/ucf101/split_2/test/ApplyEyeMakeup/

ls: cannot access '/home/jefelitman/DataSets/ucf101/split_2/test/ApplyEyeMakeup/': No such file or directory
